In [1]:
import os
import skimage
from skimage import io, transform
import math
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D,Lambda,add,Reshape,concatenate, BatchNormalization,GlobalAveragePooling2D, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
from keras.regularizers import l2
%matplotlib inline

import keras.backend as K
K.set_learning_phase(1)

Using TensorFlow backend.


In [9]:
def grouped_convolution_block(input1, grouped_channels, cardinality, strides):
    init = input1
    group_list = []
    for c in range(cardinality):
        x = Lambda(lambda z: z[:, :, :, c * grouped_channels:(c + 1) * grouped_channels])(input1)
        x = Conv2D(grouped_channels, (3, 3), padding='same', use_bias=False, strides=(strides, strides), kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(x)
        group_list.append(x)
    group_merge = concatenate(group_list, axis=-1)
    x = BatchNormalization(axis=-1)(group_merge)
    x = Activation('relu')(x)
    return x

def se_bottleneck_block(input1, filters, cardinality=32, strides=1, r=16):
    init = input1

    grouped_channels = int(filters / cardinality)
    if init._keras_shape[-1] != 2 * filters:
        init = Conv2D(filters * 2, (1, 1), padding='same', strides=(strides, strides),use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(init)
        init = BatchNormalization(axis=-1)(init)

    X = Conv2D(filters, (1, 1), padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(input1)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)

    X = grouped_convolution_block(X, grouped_channels, cardinality, strides)

    X = Conv2D(filters * 2, (1, 1), padding='same', use_bias=False, kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(X)
    X = BatchNormalization(axis=-1)(X)

    X_increase = X
    X = GlobalAveragePooling2D(data_format='channels_last')(X)
    X = Reshape([1,1,filters * 2])(X)
    X = Conv2D(filters = filters * 2 // r, kernel_size = (1, 1))(X)
    X = Activation('relu')(X)
    X = Conv2D(filters = filters * 2, kernel_size = (1, 1))(X)
    X = Activation('sigmoid')(X)
    X = layers.add([keras.layers.multiply([X,X_increase]),init])
    X = Activation('relu')(X)
    return X

#ResNeXt 50: depth = [3,4,6,3]
#ResNeXt 101: depth =[3,4,23,3]
#ResNeXt 152: depth =[3,8,36,3]
def SE_ResNeXt(input_shape = (224, 224,3), depth=[3,8,36,3], cardinality=32, width=4, r=16):
    X_input = Input(input_shape)
    X = Conv2D(64, (7, 7), padding='same', use_bias=False, strides=(2, 2), kernel_initializer='he_normal', kernel_regularizer=l2(5e-4))(X_input)
    X = BatchNormalization(axis=-1)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(X)
    
    filters = cardinality * width
    filters_list = []
    for i in range(len(depth)):
        filters_list.append(filters)
        filters *= 2 

    for i in range(depth[0]):
        X = se_bottleneck_block(X, filters_list[0], cardinality, strides=1, r=r)

    depth = depth[1:]  
    filters_list = filters_list[1:]  

    for block_idx, n_i in enumerate(depth):
        for i in range(n_i):
            if i == 0:
                X = se_bottleneck_block(X, filters_list[block_idx], cardinality, strides=2, r=r)
            else:
                X = se_bottleneck_block(X, filters_list[block_idx], cardinality, strides=1, r=r)
    X = GlobalAveragePooling2D()(X)
    X = Dense(2, use_bias=False, activation='softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='SE-ResNeXt')

    return model

In [10]:
model = SE_ResNeXt()

In [11]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1235 (Conv2D)            (None, 112, 112, 64) 9408        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_113 (BatchN (None, 112, 112, 64) 256         conv2d_1235[0][0]                
__________________________________________________________________________________________________
activation_175 (Activation)     (None, 112, 112, 64) 0           batch_normalization_113[0][0]    
__________________________________________________________________________________________________
max_poolin

__________________________________________________________________________________________________
lambda_1344 (Lambda)            (None, 28, 28, 8)    0           activation_211[0][0]             
__________________________________________________________________________________________________
conv2d_1491 (Conv2D)            (None, 28, 28, 8)    576         lambda_1313[0][0]                
__________________________________________________________________________________________________
conv2d_1492 (Conv2D)            (None, 28, 28, 8)    576         lambda_1314[0][0]                
__________________________________________________________________________________________________
conv2d_1493 (Conv2D)            (None, 28, 28, 8)    576         lambda_1315[0][0]                
__________________________________________________________________________________________________
conv2d_1494 (Conv2D)            (None, 28, 28, 8)    576         lambda_1316[0][0]                
__________

__________________________________________________________________________________________________
conv2d_1681 (Conv2D)            (None, 14, 14, 16)   2304        lambda_1482[0][0]                
__________________________________________________________________________________________________
conv2d_1682 (Conv2D)            (None, 14, 14, 16)   2304        lambda_1483[0][0]                
__________________________________________________________________________________________________
conv2d_1683 (Conv2D)            (None, 14, 14, 16)   2304        lambda_1484[0][0]                
__________________________________________________________________________________________________
conv2d_1684 (Conv2D)            (None, 14, 14, 16)   2304        lambda_1485[0][0]                
__________________________________________________________________________________________________
conv2d_1685 (Conv2D)            (None, 14, 14, 16)   2304        lambda_1486[0][0]                
__________

concatenate_53 (Concatenate)    (None, 14, 14, 512)  0           conv2d_1888[0][0]                
                                                                 conv2d_1889[0][0]                
                                                                 conv2d_1890[0][0]                
                                                                 conv2d_1891[0][0]                
                                                                 conv2d_1892[0][0]                
                                                                 conv2d_1893[0][0]                
                                                                 conv2d_1894[0][0]                
                                                                 conv2d_1895[0][0]                
                                                                 conv2d_1896[0][0]                
                                                                 conv2d_1897[0][0]                
          

__________________________________________________________________________________________________
lambda_1912 (Lambda)            (None, 14, 14, 16)   0           activation_301[0][0]             
__________________________________________________________________________________________________
lambda_1913 (Lambda)            (None, 14, 14, 16)   0           activation_301[0][0]             
__________________________________________________________________________________________________
lambda_1914 (Lambda)            (None, 14, 14, 16)   0           activation_301[0][0]             
__________________________________________________________________________________________________
lambda_1915 (Lambda)            (None, 14, 14, 16)   0           activation_301[0][0]             
__________________________________________________________________________________________________
lambda_1916 (Lambda)            (None, 14, 14, 16)   0           activation_301[0][0]             
__________

__________________________________________________________________________________________________
multiply_67 (Multiply)          (None, 14, 14, 1024) 0           activation_339[0][0]             
                                                                 batch_normalization_215[0][0]    
__________________________________________________________________________________________________
add_67 (Add)                    (None, 14, 14, 1024) 0           multiply_67[0][0]                
                                                                 activation_335[0][0]             
__________________________________________________________________________________________________
activation_340 (Activation)     (None, 14, 14, 1024) 0           add_67[0][0]                     
__________________________________________________________________________________________________
conv2d_2427 (Conv2D)            (None, 14, 14, 512)  524288      activation_340[0][0]             
__________

__________________________________________________________________________________________________
conv2d_2708 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2397[0][0]                
__________________________________________________________________________________________________
conv2d_2709 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2398[0][0]                
__________________________________________________________________________________________________
conv2d_2710 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2399[0][0]                
__________________________________________________________________________________________________
conv2d_2711 (Conv2D)            (None, 14, 14, 16)   2304        lambda_2400[0][0]                
__________________________________________________________________________________________________
concatenate_75 (Concatenate)    (None, 14, 14, 512)  0           conv2d_2680[0][0]                
          

__________________________________________________________________________________________________
lambda_2653 (Lambda)            (None, 7, 7, 32)     0           activation_416[0][0]             
__________________________________________________________________________________________________
lambda_2654 (Lambda)            (None, 7, 7, 32)     0           activation_416[0][0]             
__________________________________________________________________________________________________
lambda_2655 (Lambda)            (None, 7, 7, 32)     0           activation_416[0][0]             
__________________________________________________________________________________________________
lambda_2656 (Lambda)            (None, 7, 7, 32)     0           activation_416[0][0]             
__________________________________________________________________________________________________
conv2d_2969 (Conv2D)            (None, 7, 7, 32)     9216        lambda_2625[0][0]                
__________

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train, y_train,epochs = 900, batch_size=32,validation_data=(x_test,y_test), shuffle = True)